## Neural Network trained on more data

In [1]:
import numpy as np
import pandas as pd
import json
import keras
import keras.preprocessing.text as kpt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

seed = 123
max_words = 5000 # only work with the 5000 most popular words found in our dataset

Using TensorFlow backend.


In [2]:
negative_df = pd.read_csv("fakenews_big.csv", dtype='str')
# negative_df2 = pd.read_csv("fakenews_big2.csv", dtype='str')

positive_df = pd.read_csv("radiocanada.csv", dtype='str')
positive_df.columns = positive_df.ix[0]
positive_df = positive_df[1:]
positive_df = positive_df.sample(n=(negative_df.shape[0]), random_state=seed, axis=0)

print(positive_df.shape)
print(negative_df.shape)
# print(negative_df2.shape)


X_positive = positive_df["text"]
y_positive = pd.DataFrame(np.ones(positive_df.shape[0]))

X_negative = negative_df["text"]
y_negative = pd.DataFrame(np.zeros(negative_df.shape[0]))

# X_negative2 = negative_df2["text"]
# y_negative2 = pd.DataFrame(np.zeros(negative_df2.shape[0]))


train_x = pd.concat([X_positive, X_negative], axis=0)
train_y = pd.concat([y_positive, y_negative], axis=0)

train_x = train_x.as_matrix().astype('str')

print("X Shape:", train_x.shape)
print("y shape:", train_y.shape)

(2436, 7)
(2436, 6)
X Shape: (4872,)
y shape: (4872, 1)


In [3]:
# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary2.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

In [16]:
len(allWordIndices[5])

198

In [17]:
# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y = keras.utils.to_categorical(train_y, 2)

In [50]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [51]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [52]:
model.fit(train_x, train_y,
  batch_size=32,
  epochs=10,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

Train on 59078 samples, validate on 6565 samples
Epoch 1/10
59078/59078 [==============================] - 53s 899us/step - loss: 6.9127e-04 - acc: 0.9998 - val_loss: 5.6327 - val_acc: 0.6289
Epoch 2/10
59078/59078 [==============================] - 53s 895us/step - loss: 1.4687e-06 - acc: 1.0000 - val_loss: 5.7882 - val_acc: 0.6289
Epoch 3/10
59078/59078 [==============================] - 52s 887us/step - loss: 6.4790e-07 - acc: 1.0000 - val_loss: 5.8748 - val_acc: 0.6289
Epoch 4/10
59078/59078 [==============================] - 53s 896us/step - loss: 2.9951e-07 - acc: 1.0000 - val_loss: 5.9284 - val_acc: 0.6289
Epoch 5/10
59078/59078 [==============================] - 53s 890us/step - loss: 1.7224e-07 - acc: 1.0000 - val_loss: 5.9494 - val_acc: 0.6289
Epoch 6/10
59078/59078 [==============================] - 53s 892us/step - loss: 1.5389e-07 - acc: 1.0000 - val_loss: 5.9676 - val_acc: 0.6289
Epoch 7/10
59078/59078 [==============================] - 53s 895us/step - loss: 1.3183e-07 -

In [53]:
# Save model

model_json = model.to_json()
with open('model2.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model2.h5')